In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 13 16:23:58 2023

@author: partha.ghosh
"""

import pandas as pd 
import numpy as np 
import math

class AnalysdeDf (object): 
    
    _my_pd_data_img = None 
    _my_pd_data = None 
    _my_pd_zip_code = None
    
    # Filename: Destination change information in disk
    _my_dst_fname = None 
    
    def GetImgDataFrame(self): 
        return self._my_pd_data_img 
    
    def GetDataFrame(self): 
        return self._my_pd_data 
    
    def GetZipFrame(self):
        return self._my_pd_zip_code
    
    def __init__(self, dataFName,zipFname, changeFname):
        my_pd_data = pd.read_csv(dataFName)
        self._my_pd_data = my_pd_data
        self._my_pd_data_img =pd.DataFrame(data=None, 
                            columns=my_pd_data.columns, index=my_pd_data.index)
        self._my_pd_zip_code = pd.read_excel(zipFname) 
        self._my_change_fname = changeFname
        pass
    
    def ValidateZip(self, zipCode, cityName):
        match_flag=False
        pd_zip = self._my_pd_zip_code
        final_cities = []
        #row_indices = pd_zip.index[pd_zip["zip"]==zipCode].tolist()
        qstring = 'zip =='+str(zipCode)
        row_matches = pd_zip.query(qstring)
        if (len(row_matches)==0):
            match_flag=False
        else:
            p = row_matches["primary_city"].tolist()[0]
            if (not isinstance(p,str)): 
                p = []
            else:
                p=[" ".join(y.split()) for y in p.split(',')]
                
            a = row_matches["acceptable_cities"].tolist()[0]
            if (not isinstance(a,str)): 
                a = []
            else:
                a=[" ".join(y.split()) for y in a.split(',')]

            u = row_matches["acceptable_cities"].tolist()[0]
            if (not isinstance(u,str)): 
                u = []
            else:
                u = [" ".join(y.split()) for y in u.split(',')]

            final_cities = p + a
            unacpt_cities = u
            if (cityName in final_cities):
                match_flag=True
        return(match_flag, final_cities, unacpt_cities)
         

    def SanitizeDataRecord(self):
        num_zip_corrections = 0 
        my_pd_data = self._my_pd_data
        my_pd_data_img = self._my_pd_data_img        
        num_recs = len(my_pd_data)
        for row_index in range(num_recs):
            
            # Sanitize ZIP/CITY 
            pd_col_org_zip = my_pd_data.iloc[row_index]['ZIP5']
            pd_col_org_city = my_pd_data.iloc[row_index]['CITY']
            pd_col_org_city = ' '.join(pd_col_org_city.split())  
            (match_flag,desirable_cities,unacpt_cities) = \
                        self.ValidateZip(pd_col_org_zip,pd_col_org_city)
            if (not match_flag):
                num_zip_corrections += 1 
                my_pd_data_img.iloc[row_index]['CITY'] = desirable_cities
                print(f"Record[{row_index}]: Actual ZIP/CITY: {pd_col_org_zip}/{pd_col_org_city}\tExpected CITY:{desirable_cities}")
        return (num_recs, num_zip_corrections)
        pass 
    
    def RowStatistics(self): 
        # List of columns 
        my_pd = self._my_pd_data
        colnames = my_pd.columns 
        num_rows = len(my_pd) 
        for arow in range(num_rows): 
            pass 
        output = self._my_pd_data.describe(include='all')
        return(output)
    
    
    def DescriptiveStatistics(self): 
        # List of columns 
        my_pd = self._my_pd_data
        colnames = my_pd.columns 
        for acol in colnames: 
            tot_elems = len(my_pd[acol])
            na_elems = my_pd[acol].isna().sum();
            p_na_elems=100.0*(na_elems/tot_elems)
            null_elems = my_pd[acol].isnull().sum()
            p_null_elems=100.0*(null_elems/tot_elems)
            nz_elems = tot_elems-(na_elems+null_elems) 
            p_nz_elems=100.0*(nz_elems/tot_elems)
            print(f"[{acol}] ====>")
            print(f"\t\tNZ:{nz_elems}({p_nz_elems}%)") 
            print(f"\t\tNA:{na_elems}({p_na_elems}%)") 
            print(f"\t\tNULL:{null_elems}({p_null_elems}%)") 
        output = self._my_pd_data.describe(include='all')
        return(output)
    
    def WriteChangesToDisk(self): 
        # Open wwritable disk fname 
        fname = self._my_change_fname
        
        # Dump pd frame into disk 
        pd_frame = self._my_pd_data_img
        pd_frame.to_excel(fname)
        pass 
    
    pass

    


fname = "C:\\ParthaGhosh\\Personal\\Technical\\new_Data\\MachineLearning\\Dataset\\Iris\\iris\\bezdekIris.data"
zip_fname = "C:\\ParthaGhosh\\Personal\\Technical\\new_Data\\MachineLearning\\Dataset\\Zipcode\\zip_code_database.xls"
change_fname="C:\\ParthaGhosh\\Personal\\Technical\\new_Data\\MachineLearning\\Dataset\\Iris\\iris\\change_fname.xlsx"

dfobj = AnalysdeDf(fname,zip_fname,change_fname)
dfobj.DescriptiveStatistics()
(num_recs, num_zip_corrections) = dfobj.SanitizeDataRecord()
p_zip_corrections = 100.0*(num_zip_corrections/num_recs)
print(f"Num_Recs: {num_recs}\t Num_Zip_Corrections:{num_zip_corrections}")
print(f"Pcnt ZIP Corrcetions: {p_zip_corrections}")
row_stats = dfobj.RowStatistics()
dfobj.WriteChangesToDisk()


